<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Analisis de sentimiento 

Guillermo Andres Cardona
cod:616202023

In [85]:
import pandas as pd
import tweepy
import regex
import emoji

import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from textblob import TextBlob 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

from flair.models import TextClassifier
from flair.data import Sentence

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


In [86]:
# Leer las llaves
keys = pd.read_csv("G:/token/tweet.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)



In [87]:
tweets = tweepy.Cursor(api.search,
                       q=['paro en colombia -RT'],#RT quitar los retwteados
                       lang='es',
                       #geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)


# Objeto generador
t = [tweet for tweet in tweets]


In [88]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    #temp.append('Washington')
    #temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'id', 'link']

In [89]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['tweets', 'date', 'id', 'link']] = data.apply(text_clean, axis=1, result_type='expand')

In [81]:
data['sentvader'] = data['tweets'].apply(lambda valor: vader.polarity_scores(valor)['compound'])
data['senttextblob'] = data['tweets'].apply(lambda valor: TextBlob(valor).sentiment.polarity)
pd.set_option('display.max_colwidth', None)
data[['tweets', 'sentvader']].sort_values('sentvader')

,tweets,sentvader
49,¿Colombia está en crisis por el paro o en paro por la crisis?,-0.8481
67,Que viva el paro nacional sin vandalismo y sin que se maten. en Colombia,-0.8020
23,"Mi compañera la boba hpta dizque ""Hay que volver a clases, no ven que es nuestro futuro y entre más paro más atrás quedamos"" amiga, estamos en Colombia, el país del NO FUTURO. Tanto afán para salir a trabajar en un call center porque oportunidades no hay.",-0.7456
13,"Urrejola aseveró que la situación en Colombia es preocupante, porque la protesta es un derecho que debe ser garantizado por el Estado, cosa que no se ha visto en el reciente paro nacional.\nTipo su país NO los toquen dejen que destruyan a bichito malo",-0.7326
24,"El paro es consecuencia de las migajas de país que quedan en 200 años y donde la gente habla sin conocer la situación del país en total. Colombia no es Bogotá, Medellín y Cali son 32 departamentos y todos tienen abandono estatal",-0.7251
...,...,...
43,"Por favor en que país se encuentra usted, ese no es Colombia, por algo el presidente tiene un 78% de desfavorable, quiere decir que ese 78 apoya el paro. Aterricé de dónde anda. Otra que vive el Polombia",0.1280
3,Los que apoyaron al dizque ministro de defensa llevarán en sus negras conciencias las desparaciones que ha dejado el paro todo por culpa de este títere que dice llamarse presidente y del titiretero mayor responsable de todas las desparaciones en colombia,0.3400
22,Es de resaltar que Adolfo Pérez Esquivel es parte de la Comisión Ética por la Verdad que funciona en Colombia y que durante más de 10 años ha venido recogiendo testimonios de la víctimas y procesos comunitarios en el país,0.3400
35,"Será que con alcohol y fiesta se nos va a olvidar que nos están masacrando en Colombia, que estamos en paro, que violan los derechos de los Colombianos a diestra y siniestra",0.4767


In [82]:
def maspos(dataf, datomostrar, cant):
    datomp = dataf.sort_values(datomostrar, ascending = False)
    print(datomp.head(cant)) 
    print()
    print("*********************************************************************")
    print()
    print(datomp.tail(cant)) 

In [83]:
maspos(data, 'sentvader', 1)#mas positivo y mas negativo

                                                             tweets date  id  \
54  Los del Tal Paro en  después de conocer...que el   \nJajaja 😂🤣    []  []   

   link  sentvader  senttextblob  
54   []     0.7269           0.0  

*********************************************************************

                                                           tweets date  id  \
49  ¿Colombia está en crisis por el paro o en paro por la crisis?   []  []   

   link  sentvader  senttextblob  
49   []    -0.8481           0.0  


In [76]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
frase = Sentence(data['tweets'])

classifier.predict(frase)
frase.labels

l[0].value, l[0].score


2021-05-27 14:52:09,520 loading file C:\Users\Memo\.flair\models\sentiment-en-mix-distillbert_4.pt


TypeError: expected string or bytes-like object

In [84]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)